<a href="https://colab.research.google.com/github/sudheerranjan/sudheerranjan.github.io/blob/master/Rossmann_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 2.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=8faae5096091e7f77535d8d21b71c6c974d70e2c8e3168d91e53c2411ba8db8a
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [3]:
! mkdir ~/.kaggle # /content
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download rossmann-store-sales

  0% 0.00/6.99M [00:00<?, ?B/s]
100% 6.99M/6.99M [00:00<00:00, 89.7MB/s]


In [5]:
!unzip "/content/rossmann-store-sales.zip" -d "/content/dataset/"

Archive:  /content/rossmann-store-sales.zip
  inflating: /content/dataset/sample_submission.csv  
  inflating: /content/dataset/store.csv  
  inflating: /content/dataset/test.csv  
  inflating: /content/dataset/train.csv  


## Imports

In [1]:
import pandas as pd             # Read Data
import matplotlib.pyplot as plt # Plotting
import seaborn as sns           # Plotting
import numpy as np              #Scientific computations

##Load Data to DataFrame

In [25]:
train_data = pd.read_csv('/content/dataset/train.csv',parse_dates=['Date'],squeeze=True, low_memory=False)
train_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [26]:
store_data = pd.read_csv('/content/dataset/store.csv')
store_data.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


###Extract Date features

In [27]:
train_data['day'] = train_data['Date'].dt.day
train_data['week'] = train_data['Date'].dt.isocalendar().week
train_data['month'] = train_data['Date'].dt.month
train_data['quarter'] = train_data['Date'].dt.quarter
train_data['year'] = train_data['Date'].dt.year
train_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,day,week,month,quarter,year
0,1,5,2015-07-31,5263,555,1,1,0,1,31,31,7,3,2015
1,2,5,2015-07-31,6064,625,1,1,0,1,31,31,7,3,2015
2,3,5,2015-07-31,8314,821,1,1,0,1,31,31,7,3,2015
3,4,5,2015-07-31,13995,1498,1,1,0,1,31,31,7,3,2015
4,5,5,2015-07-31,4822,559,1,1,0,1,31,31,7,3,2015


###Basic Observations from the datasets

In [9]:
print(f'Number of rows and features in train_data : {train_data.shape}')
print(f'Number of rows and features in store_data : {store_data.shape}')
print(f'List of features in train_data : {list(train_data.columns)}')
print(f'List of features in store_data : {list(store_data.columns)}')

Number of rows and features in train_data : (1017209, 14)
Number of rows and features in store_data : (1115, 10)
List of features in train_data : ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'day', 'week', 'month', 'quarter', 'year']
List of features in store_data : ['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


In [10]:
print(f'Number of unique stores in the train_data: {len(list(train_data.Store.unique()))}')
print(f'Number of unique stores in the store_data: {len(list(store_data.Store.unique()))}')

Number of unique stores in the train_data: 1115
Number of unique stores in the store_data: 1115


In [11]:
print(f'train_data contains data from {min(train_data.Date)} to {max(train_data.Date)} with a total of {max(train_data.Date) - min(train_data.Date)} in between')

train_data contains data from 2013-01-01 00:00:00 to 2015-07-31 00:00:00 with a total of 941 days 00:00:00 in between


In [12]:
train_data.nunique()

Store             1115
DayOfWeek            7
Date               942
Sales            21734
Customers         4086
Open                 2
Promo                2
StateHoliday         4
SchoolHoliday        2
day                 31
week                52
month               12
quarter              4
year                 3
dtype: int64

In [15]:
store_data.nunique()

Store                        1115
StoreType                       4
Assortment                      3
CompetitionDistance           654
CompetitionOpenSinceMonth      12
CompetitionOpenSinceYear       23
Promo2                          2
Promo2SinceWeek                24
Promo2SinceYear                 7
PromoInterval                   3
dtype: int64

###Missing Values in the Dataset

In [13]:
#Check for missing values in train_data
train_data.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
day              0
week             0
month            0
quarter          0
year             0
dtype: int64

In [14]:
#Check for missing values in train_data
store_data.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [28]:
train_data[(train_data['Open']==0) & (train_data['Sales']==0)].shape

(172817, 14)

In [29]:
train_data[(train_data['Open']==1) & (train_data['Sales']==0)].shape

(54, 14)

In [30]:
train_data = train_data[(train_data['Open']!=0) & (train_data['Sales']!=0)]

In [31]:
train_data.shape

(844338, 14)

In [32]:
store_data['CompetitionDistance'].fillna(store_data['CompetitionDistance'].median(), inplace=True)

In [33]:
store_data.fillna(0, inplace=True)

### Merge Datasets

In [34]:
train_store_data = train_data.merge(store_data, on='Store',how='left')
train_store_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,day,week,month,quarter,year,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,31,31,7,3,2015,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,31,31,7,3,2015,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,31,31,7,3,2015,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,31,31,7,3,2015,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,31,31,7,3,2015,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [35]:
#Check for missing values in train_data
train_store_data.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
day                          0
week                         0
month                        0
quarter                      0
year                         0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [ ]:
# https://stackoverflow.com/questions/47230817/plotly-notebook-mode-with-google-colaboratory